In [1]:
# flags.py
import os
import time
import sys
import requests

POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()
BASE_URL = 'http://flupy.org/data/flags'
DEST_DIR = '/Users/long/output'

def save_flag(img, filename):
    path = os.path.join(DEST_DIR, filename) 
    with open(path, 'wb') as fp:
        fp.write(img)
        
def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower()) 
    resp = requests.get(url)
    return resp.content


def show(text): 
    print(text, end=' ') 
    sys.stdout.flush()

def download_many(cc_list): 
    for cc in sorted(cc_list):
        image = get_flag(cc)
        show(cc)
        save_flag(image, cc.lower() + '.gif')
    return len(cc_list)


def main(download_many):
    t0 = time.time()
    count = download_many(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags downloaded in {:.2f}s' 
    print(msg.format(count, elapsed))

# main(download_many)

In [5]:
# flags_threadpool.py
from concurrent import futures

MAX_WORKERS = 20


def download_one(cc):
    time.sleep(3)
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif') 
    return cc


def download_many(cc_list):
    workers = min(MAX_WORKERS, len(cc_list))
    with futures.ThreadPoolExecutor(workers) as executor:
        res = executor.map(download_one, sorted(cc_list)) 
    
    return len(list(res))
 
    
main(download_many)

BD ID BR VN DE FR EG PK IN IR ET MX US CN NG JP TR PH CD RU 
20 flags downloaded in 3.80s


In [11]:
# replacing executor.map with execu tor.submit and futures.as_completed in the download_many function

def download_many(cc_list):
    cc_list = cc_list[:5]
    with futures.ThreadPoolExecutor(max_workers=3) as executor:
        to_do = []
        for cc in sorted(cc_list):
            future = executor.submit(download_one, cc)
            to_do.append(future)
            msg = 'Scheduled for {}: {}' 
            print(msg.format(cc, future))
            
    results = []
    print('herer')  # 为啥不是立刻输出？
    for future in futures.as_completed(to_do):
        res = future.result()
        msg = '{} result: {!r}' 
        print(msg.format(future, res)) 
        results.append(res)
    
    return len(results)

main(download_many)



Scheduled for BR: <Future at 0x10e12d5f8 state=running>
Scheduled for CN: <Future at 0x10e12d898 state=running>
Scheduled for ID: <Future at 0x10dfe6208 state=running>
Scheduled for IN: <Future at 0x10dfea4a8 state=pending>
Scheduled for US: <Future at 0x10dfea358 state=pending>
BR CN ID IN US herer
<Future at 0x10e12d5f8 state=finished returned str> result: 'BR'
<Future at 0x10dfea358 state=finished returned str> result: 'US'
<Future at 0x10dfea4a8 state=finished returned str> result: 'IN'
<Future at 0x10e12d898 state=finished returned str> result: 'CN'
<Future at 0x10dfe6208 state=finished returned str> result: 'ID'

5 flags downloaded in 12.16s


sdfsdf

In [14]:
def download_many(cc_list):
    with futures.ProcessPoolExecutor() as executor:
        res = executor.map(download_one, sorted(cc_list)) 
    
    return len(list(res))
 
    
main(download_many)

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [22]:
# demo_executor_map.py: Simple demonstration of the map method of ThreadPoolExecutor.
from time import sleep, strftime 
from concurrent import futures

def display(*args): 
    print(strftime('[%H:%M:%S]'), end=' ') 
    print(*args)
    
def loiter(n):
    msg = '{}loiter({}): doing nothing for {}s...' 
    display(msg.format('\t'*n, n, n))
    sleep(n)
    msg = '{}loiter({}): done.' 
    display(msg.format('\t'*n, n))
    return n * 10

def main():
    display('Script starting.')
    executor = futures.ThreadPoolExecutor(max_workers=3) 
    results = executor.map(loiter, range(6)) 
    display('results:', results) # . display('Waiting for individual results:')
    for i, result in enumerate(results):
        display('result {}: {}'.format(i, result))

main()

[19:39:10] Script starting.
[19:39:10] loiter(0): doing nothing for 0s...
[19:39:10] loiter(0): done.
[19:39:10] 	loiter(1): doing nothing for 1s...
[19:39:10] 		loiter(2): doing nothing for 2s...
[19:39:10] results: <generator object Executor.map.<locals>.result_iterator at 0x10e1956d8>
[19:39:10] result 0: 0
[19:39:10] 			loiter(3): doing nothing for 3s...
[19:39:11] 	loiter(1): done.
[19:39:11] 				loiter(4): doing nothing for 4s...
[19:39:11] result 1: 10
[19:39:12] 		loiter(2): done.
[19:39:12] 					loiter(5): doing nothing for 5s...
[19:39:12] result 2: 20
[19:39:13] 			loiter(3): done.
[19:39:13] result 3: 30
[19:39:15] 				loiter(4): done.
[19:39:15] result 4: 40
[19:39:17] 					loiter(5): done.
[19:39:17] result 5: 50
